In [1]:
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [2]:
import pandas as pd
import re

df = pd.read_csv("호주_세관_v2(new20).csv")

df_title = list(df['title'])
df_text = list(df['text'])
df_add = []

for i in range(len(df)):
    df_add.append(df_title[i]+""+df_text[i])

df_keyword = pd.read_csv("호주_번역_100.csv", index_col = False)

In [3]:
#고급 차량 관련 문서로 테스트 진행
doc = df_add[1]


In [4]:
import string
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

#구두점 제거
doc1 = "".join([i for i in doc if i not in string.punctuation]).strip()
print(len(doc1))

#숫자 제거
doc2 = "".join([i for i in doc1 if not i.isdigit()])
print(len(doc2))

#월 제거
month = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august',
        'september', 'october', 'november', 'december', 'jan', 'feb', 'mar', 'apr',
         'may', 'jun','jul', 'aug', 'sep', 'oct', 'nov', 'dec']   
    
doc3 = " ".join([i for i in doc2.split() if i not in month])
print(len(doc3))

# word_tokens = word_tokenize(doc2)

#동사 명사
# word_tokens = pos_tag(word_tokenize(doc))
# word_tokens = [t[0] for t in word_tokens if t[1] == "NN"]

1815
1710
1653


In [5]:
n_gram_range = (2,2)
stop_words = "english"

count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc3])
candidates = count.get_feature_names_out()

print('bigram 개수 :',len(candidates))
print('bigram 출력 :',candidates)

bigram 개수 : 125
bigram 출력 : ['abfgovau amendments' 'additional duty' 'additional note'
 'alterations notice' 'amendments online' 'applied implementation'
 'applies goods' 'apply refund' 'attachment notice' 'attachment online'
 'australian notice' 'available attachment' 'available current'
 'belarus currently' 'car tax' 'cargo ics' 'cargo new' 'cent applied'
 'certain electric' 'chapter new' 'circumstance eb'
 'classifications available' 'classifications commence'
 'classifications entered' 'commence retrospectively'
 'consistent australian' 'consumption lct' 'consumption time'
 'current working' 'currently additional' 'customs duty'
 'directed tradepolicyabfgovau' 'duty applies' 'duty cent' 'duty certain'
 'duty electric' 'duty free' 'eb quoted' 'efficient luxury'
 'efficient vehicles' 'electric vehicles' 'entered home' 'entered ics'
 'entered integrated' 'financial year' 'following publication' 'free rate'
 'fuel efficient' 'fuelcell vehicles' 'goods entered' 'goods meet'
 'home consu

In [6]:
#문서 전체와 문서에서 추출한 키워드 수치화
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([doc]) #전체 문서
candidate_embeddings = model.encode(candidates)#추출한 bigram

In [7]:
#문서와 가장 유사한 키워드 추출
doc_bigram_keywords = []

top_n = 20
distances = cosine_similarity(doc_embedding, candidate_embeddings)
doc_bigram_keyword = [candidates[index] for index in distances.argsort()[0][-top_n:]]
doc_bigram_keywords.append(doc_bigram_keyword)

print(len(doc_bigram_keywords))
print(doc_bigram_keywords)

1
[['tariff available', 'removal customs', 'fuelcell vehicles', 'vehicles australian', 'cargo new', 'duty electric', 'russia belarus', 'vehicles hydrogen', 'certain electric', 'amendments online', 'propose tariff', 'hydrogen fuelcell', 'electric vehicles', 'hybrid vehicles', 'revised tariff', 'tariff alterations', 'new tariff', 'car tax', 'online tariff', 'tax threshold']]


In [14]:
#문서와 가장 유사한 키워드 벡터화
doc_bigram_embedding = []
for keyword in doc_bigram_keywords:
    doc_bigram_embedding.append(model.encode(keyword))

In [15]:
#keyword : 불러온 호주 키워드 중 번역 Column 추출
keyword = list(df_keyword["번역"])
keyword_embedding = []

for ele in keyword:
    keyword_embedding.append(model.encode(ele))

print(len(keyword_embedding))

102


In [16]:
#doc_bigram_keywords -> doc_bigram_embedding
#keyword -> keyword_embedding

In [17]:
#문서 키워드(doc_bigram_embedding)와 키워드 100(keyword_embedding) 유사도 비교 
bigram_key_result = []
keyword_key_result = []
cosine_distances = []

for index, bigram in enumerate(doc_bigram_embedding):
    bigram_result = []
    key_result=[]
    cosine_distance = []
    for i in range(len(bigram)):
        for j in range(len(keyword_embedding)):
            distances = cosine_similarity([bigram[i]],[keyword_embedding[j]])
            if distances[0][0] > 0.75:
                bigram_result.append(doc_bigram_keywords[index][i])
                key_result.append(keyword[j])
                cosine_distance.append(distances)

    bigram_key_result.append(bigram_result)
    keyword_key_result.append(key_result)
    cosine_distances.append(cosine_distance)

In [18]:
df_check = pd.DataFrame()

print(bigram_key_result)
print(keyword_key_result)

for i in range(len(bigram_key_result)):
    bigram = pd.Series(bigram_key_result[i])
    keyword = pd.Series(keyword_key_result[i])
    distance = pd.Series(cosine_distances[i])
    
    df_check['bigram'] = bigram
    df_check['keyword'] = keyword
    df_check['distance'] = distance

df_check

[['fuelcell vehicles', 'fuelcell vehicles', 'duty electric', 'car tax']]
[['LNG (liquid natural gas)', 'propane', 'propane', 'car']]


,bigram,keyword,distance
0,fuelcell vehicles,LNG (liquid natural gas),[[0.7554338]]
1,fuelcell vehicles,propane,[[0.7691965]]
2,duty electric,propane,[[0.76044923]]
3,car tax,car,[[0.75818545]]
